In [1]:
from itertools import chain
from functools import reduce
import pandas as pd
import numpy as np
import os

# 民宿价格预测, 数据预处理

## 一、数据读取

In [2]:
data_path = r'C://Users//86199//Documents//Datas//hey_whale//C1'
os.chdir(data_path)
os.listdir()

['提交样例.csv', '测试集.csv', '训练集.csv']

In [3]:
train = pd.read_csv('训练集.csv').set_index('数据ID')
test = pd.read_csv('测试集.csv').set_index('数据ID')
df = pd.concat([train, test])
df.head()

,容纳人数,便利设施,洗手间数量,床的数量,床的类型,卧室数量,取消条款,所在城市,清洁费,首次评论日期,...,最近评论日期,维度,经度,民宿周边,评论个数,房产类型,民宿评分,房型,邮编,价格
数据ID,,,,,,,,,,,,,,,,,,,,,
train_0,4,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.5,3.0,4,2.0,0,3,0,2015-05-07,...,2016-06-26,34.109039,-118.273390,Los Feliz,12,17,97.0,0,90027,64.918531
train_1,2,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",1.0,1.0,4,1.0,2,4,1,2016-07-02,...,2016-07-31,40.812897,-73.919163,Mott Haven,6,0,87.0,0,10454,54.918531
train_2,4,"{TV,""Air conditioning"",Kitchen,Heating,""Smoke ...",1.0,2.0,4,0.0,2,4,1,2017-07-01,...,2017-07-31,40.737643,-73.953309,Greenpoint,4,0,80.0,0,11222,73.219281
train_3,2,{},1.0,1.0,4,1.0,0,5,1,NaN,...,NaN,37.759935,-122.420558,Mission District,0,0,NaN,1,94110,64.093909
train_4,3,"{Internet,""Wireless Internet"",""Air conditionin...",1.0,1.0,4,1.0,1,4,1,2014-04-30,...,2016-05-22,40.683363,-73.949490,Bedford-Stuyvesant,16,0,99.0,0,11216,68.454901


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74111 entries, train_0 to test_14822
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   容纳人数         74111 non-null  int64  
 1   便利设施         74111 non-null  object 
 2   洗手间数量        73911 non-null  float64
 3   床的数量         73980 non-null  float64
 4   床的类型         74111 non-null  int64  
 5   卧室数量         74020 non-null  float64
 6   取消条款         74111 non-null  int64  
 7   所在城市         74111 non-null  int64  
 8   清洁费          74111 non-null  int64  
 9   首次评论日期       58247 non-null  object 
 10  房主是否有个人资料图片  73923 non-null  object 
 11  房主身份是否验证     73923 non-null  object 
 12  房主回复率        55812 non-null  object 
 13  何时成为房主       73923 non-null  object 
 14  是否支持随即预订     74111 non-null  int64  
 15  最近评论日期       58284 non-null  object 
 16  维度           74111 non-null  float64
 17  经度           74111 non-null  float64
 18  民宿周边         67239 non-null  object 
 19

## 二、非结构化数据, 便利设施字段分析研究

#### 2.1 单字段，单个样本研究

In [5]:
str_a = df['便利设施'][0]
print(str_a)
print(type(str_a))
list_a = str_a[1:-1].split(',')
print(list_a) # 存在翻译缺失

{TV,"Cable TV",Internet,"Wireless Internet","Air conditioning",Kitchen,"Pets live on this property",Dog(s),"Hot tub",Heating,Washer,Dryer,"Smoke detector","Fire extinguisher",Essentials,"translation missing: en.hosting_amenity_49","translation missing: en.hosting_amenity_50"}
<class 'str'>
['TV', '"Cable TV"', 'Internet', '"Wireless Internet"', '"Air conditioning"', 'Kitchen', '"Pets live on this property"', 'Dog(s)', '"Hot tub"', 'Heating', 'Washer', 'Dryer', '"Smoke detector"', '"Fire extinguisher"', 'Essentials', '"translation missing: en.hosting_amenity_49"', '"translation missing: en.hosting_amenity_50"']


#### 2.2 循环遍历提取字段信息

In [6]:
# 对每一行的便利设施字符串进行处理
list_b = [each[1:-1].split(',') for each in df['便利设施'].tolist()]
# 获取所有样本的便利设施信息
type_list = list_b[0]
for each_list in list_b[1:]:
    for each in each_list:
        type_list.append(each)

#### 2.3 主要特征——便利设施

In [7]:
type_df = pd.DataFrame(type_list, columns=['name'])
type_df.head()

,name
0,TV
1,"""Cable TV"""
2,Internet
3,"""Wireless Internet"""
4,"""Air conditioning"""


In [8]:
type_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304532 entries, 0 to 1304531
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   name    1304532 non-null  object
dtypes: object(1)
memory usage: 10.0+ MB


#### 2.3 最受欢迎的十大便利设施

In [15]:
type_nums = type_df.groupby('name').agg(value = ('name', 'count')).sort_values(by='value', ascending=False).reset_index()
# type_nums = type_nums.assign( per = type_nums['value'] / len(type_list))
type_nums.head(10)

,name,value
0,"""Wireless Internet""",71265
1,Kitchen,67526
2,Heating,67073
3,Essentials,64005
4,"""Smoke detector""",61727
5,"""Air conditioning""",55210
6,TV,52458
7,Shampoo,49465
8,Hangers,49173
9,"""Carbon monoxide detector""",47190


In [19]:
main_types = type_nums.head(10)['name'].tolist()
print(main_types)

['"Wireless Internet"', 'Kitchen', 'Heating', 'Essentials', '"Smoke detector"', '"Air conditioning"', 'TV', 'Shampoo', 'Hangers', '"Carbon monoxide detector"']
